In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder, RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display # Allows the use of display() for DataFrames

# Import supplementary visualizations code visuals.py
import visuals as vs

# Pretty display for notebooks
%matplotlib inline

In [ ]:
## load data
products = pd.read_csv('../cproducts.csv')
tender = pd.read_csv('../ctender.csv')

In [ ]:
products.head()

In [ ]:
tender.head()

In [ ]:
## fill missing values

products['promotion_description'].fillna('no_promo', inplace=True)
products['Gender'].fillna('no_gender', inplace=True)
products['State'].fillna('no_state', inplace=True)
products['PinCode'].fillna(-1, inplace=True)
products['DOB'].fillna("1", inplace=True)

In [ ]:
## convert data into numeric / float

for c in products.columns:
    lbl = LabelEncoder()
    if products[c].dtype == 'object' and c not in ['store_description','customerID','transactionDate']:
        products[c] = lbl.fit_transform(products[c])

In [ ]:
products.info()

In [ ]:
customers = products['customerID']
store_codes = products['store_code']

In [ ]:
products_new = products.drop('customerID', axis=1, inplace=True)
products_new = products.drop('transactionDate', axis=1, inplace=True)
products_new = products.drop('store_description', axis=1)

## Implementation: Selecting Samples¶

To get a better understanding of the customers and how their data will transform through the analysis, it would be best to select a few sample data points and explore them in more detail. In the code block below, we will add three indices of our choice to the indices list which will represent the customers to track. It is suggested to try different sets of samples until we obtain customers that vary significantly from one another.


In [ ]:
# TODO: Select three indices of your choice you wish to sample from the dataset
indices = [60,110,160]

# Create a DataFrame of the chosen samples
samples = pd.DataFrame(products_new.loc[indices], columns = products_new.keys()).reset_index(drop = True)
print ("Chosen samples of wholesale customers dataset:")
display(samples)


percentiles = products_new.rank(pct=True)
percentiles = 100*percentiles.round(decimals=3)
percentiles = percentiles.iloc[indices]

plt.axes().set_title("HeatMap")
sns.heatmap(percentiles, vmin=1, vmax=99, annot=True, cbar=False, square=True)

## Implementation: Feature Relevance

One interesting thought to consider is if one (or more) of the thirteen product categories is actually relevant for understanding customer purchasing. We can make this determination quite easily by training a supervised regression learner on a subset of the data with one feature removed, and then score how well that model can predict the removed feature. In the code block below, we will need to implement the following: Assign new_data a copy of the data by removing a feature of our choice using the DataFrame.drop function. Use sklearn.model_selection.train_test_split to split the dataset into training and testing sets. Use the removed feature as our target label. Set a test_size of 0.25 and set a random_state. Import a decision tree regressor, set a random_state, and fit the learner to the training data. Report the prediction score of the testing set using the regressor's score function.


In [ ]:
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

# TODO: Make a copy of the DataFrame, using the 'drop' function to drop the given feature
store_data = products_new['store_code']
new_data = products_new.drop('store_code', axis=1)

# TODO: Split the data into training and testing sets using the given feature as the target


X_train, X_test, y_train, y_test = model_selection.train_test_split(new_data, store_data, test_size=0.25, random_state=42)

# TODO: Create a decision tree regressor and fit it to the training set

regressor = DecisionTreeRegressor(random_state=42)
regressor.fit(X_train, y_train)
pred = regressor.predict(X_test)
# TODO: Report the score of the prediction using the testing set
score = r2_score(y_test,pred)
score

The reported coefficient of determination or the R squared value of store_code was 0.75 which shows a strong fit of the model to the data. This feature may not be necessary to identify customers' spending habit as this can be pretty easily deduced from the rest of the features.

In [ ]:
# Produce a scatter matrix for each pair of features in the data
pd.plotting.scatter_matrix(products_new, alpha = 0.5, figsize = (14,8), diagonal = 'kde');

In [ ]:
products_new.isnull().values.any()

In [ ]:
display(samples)

In [ ]:
products_new.info()


## Feature Transformation

In this section we will use principal component analysis (PCA) to draw conclusions about the underlying structure of the wholesale customer data. Since using PCA on a dataset calculates the dimensions which best maximize variance, we will find which compound combinations of features best describe customers.

## Implementation: PCA

We can now apply PCA to the good_data to discover which dimensions about the data best maximize the variance of features involved. In addition to finding these dimensions, PCA will also report the explained variance ratio of each dimension — how much variance within the data is explained by that dimension alone. Note that a component (dimension) from PCA can be considered a new "feature" of the space, however it is a composition of the original features present in the data.

In the code block below, we will need to implement the following:

    Import sklearn.decomposition.PCA and assign the results of fitting PCA in 6 dimensions with products_new to pca.
    Apply a PCA transformation of samples using pca.transform, and assign the results to pca_samples.



In [ ]:
from sklearn.decomposition import PCA

# TODO: Apply PCA by fitting the good data with only two dimensions
pca = PCA(n_components=6).fit(products_new)

# TODO: Transform the good data using the PCA fit above
reduced_data = pca.transform(products_new)

# TODO: Transform log_samples using the PCA fit above
pca_samples = pca.transform(samples)

# Create a DataFrame for the reduced data
#reduced_data = pd.DataFrame(reduced_data, columns = ['Dimension 1', 'Dimension 2'])

pca_results = vs.pca_results(products_new, pca)

print(pca_results['Explained Variance'].cumsum())

In [ ]:
from sklearn.decomposition import PCA

# TODO: Apply PCA by fitting the good data with only two dimensions
pca = PCA(n_components=2).fit(products_new)

# TODO: Transform the good data using the PCA fit above
reduced_data = pca.transform(products_new)

# TODO: Transform log_samples using the PCA fit above
pca_samples = pca.transform(samples)

# Create a DataFrame for the reduced data
reduced_data = pd.DataFrame(reduced_data, columns = ['Dimension 1', 'Dimension 2'])

vs.pca_results(products_new, pca)

In [ ]:
# Display sample log-data after applying PCA transformation in two dimensions
display(pd.DataFrame(np.round(pca_samples, 4), columns = ['Dimension 1', 'Dimension 2']))


## Creating clusters

GMM is a lot more flexible in terms of cluster covariance. K-means is actually a special case of GMM in which each cluster’s covariance along all dimensions approaches 0. This implies that a point will get assigned only to the cluster closest to it. With GMM, each cluster can have unconstrained covariance structure. Think of rotated and/or elongated distribution of points in a cluster, instead of spherical as in K-means. As a result, cluster assignment is much more flexible in GMM than in K-means.
## Implementation: Creating Clusters

Depending on the problem, the number of clusters that we expect to be in the data may already be known. When the number of clusters is not known a priori, there is no guarantee that a given number of clusters best segments the data, since it is unclear what structure exists in the data — if any. However, we can quantify the "goodness" of a clustering by calculating each data point's silhouette coefficient. The silhouette coefficient for a data point measures how similar it is to its assigned cluster from -1 (dissimilar) to 1 (similar). Calculating the mean silhouette coefficient provides for a simple scoring method of a given clustering.


In [ ]:
from sklearn.mixture import GaussianMixture 
from sklearn.metrics import silhouette_score

def silhouette(k):
    global clusterer, preds, centers, sample_preds
    
    # Apply your clustering algorithm of choice to the reduced data 
    clusterer = GaussianMixture(n_components=k, random_state=0)
    clusterer.fit(reduced_data)

    # Predict the cluster for each data point
    preds = clusterer.predict(reduced_data)

    # Find the cluster centers
    centers = clusterer.means_ 
    
    # Predict the cluster for each transformed sample data point
    sample_preds = clusterer.predict(pca_samples)

    # Calculate the mean silhouette coefficient for the number of clusters chosen
    score = silhouette_score(reduced_data,preds)
    return score, preds

results = pd.DataFrame(columns=['Silhouette Score'])
results.columns.name = 'Number of Clusters'    
for k in range(2,16):
    score,_ = silhouette(k) 
    results = results.append(pd.DataFrame([score],columns=['Silhouette Score'],index=[k]))

display(results)


## Cluster Visualization

Once we've chosen the optimal number of clusters for our clustering algorithm using the scoring metric above, we can now visualize the results by executing the code block below. Note that, for experimentation purposes, you are welcome to adjust the number of clusters for your clustering algorithm to see various visualizations. The final visualization provided should, however, correspond with the optimal number of clusters.


In [ ]:
#Optimum silhouette score
scores, preds = silhouette(5)

# Display the results of the clustering from implementation
vs.cluster_results(reduced_data, preds, centers, pca_samples)


## Implementation: Data Recovery

Each cluster present in the visualization above has a central point. These centers (or means) are not specifically data points from the data, but rather the averages of all the data points predicted in the respective clusters. For the problem of creating customer segments, a cluster's center point corresponds to the average customer of that segment.


In [ ]:
# Display the true centers
segments = ['Segment {}'.format(i) for i in range(0,len(centers))]
true_centers = pd.DataFrame(np.round(centers), columns = reduced_data.keys())
true_centers.index = segments
display(true_centers)

In [ ]:
# Display the predictions
for i, pred in enumerate(sample_preds):
    print ("Sample point", i, "predicted to be in Cluster", pred)

In [ ]:
## create submission files
sub2 = pd.DataFrame({'customerID':customers, 'store_code':store_codes, 'cluster':preds})

In [ ]:
np.savetxt('../subthree_new.txt', reduced_data)
sub2.to_csv('../subthree_new.csv', index=False)